In [13]:
from evaluator import SyntacticCalculator, MorphologicalCalculator, InventoryCalculator, PhonologicalCalculator, FeaturalCalculator, ScripturalCalculator, LangRankEvaluator
# typ_file = 'selection_result/imputed_laplacian_300.csv'
typ_file = 'data/URIELPlus_Union.csv'
syn = SyntacticCalculator(typ_file)
morph = MorphologicalCalculator(typ_file)
inv = InventoryCalculator(typ_file)
phon = PhonologicalCalculator(typ_file)
feat = FeaturalCalculator(typ_file)
script = ScripturalCalculator('data/URIEL_Scriptural.csv')
evaluator = LangRankEvaluator(
    calculators={
    'syntactic': syn,
    'morphological': morph,
    'inventory': inv,
    'phonological': phon,
    'featural': feat,
    'scriptural': script
    },
    iso_map_file='data/code_mapping.csv', # path to the ISO to Glottocode mapping file
)

In [14]:
features = ['syntactic', 'inventory', 'phonological', 'featural']
features += ['morphological']
# features += ['scriptural']
task_col_name = 'task_lang'
transfer_col_name = 'transfer_lang'

tasks = {
    'mt': ('BLEU', True),
    'dep': ('accuracy', True),
    'el': ('accuracy', True),
    'pos': ('accuracy', True),
    'taxi1500': ('f1_score', False),
}
for task in tasks:
    # replace/add distances in the `distance_types` columns of the dataset CSV using the corresponding calculators passed into the evaluator.
    evaluator.replace_distances(
        dataset_path=f'data/{task}.csv', # path to the task dataset (the one containing columns for task lang, transfer lang, performance)
        distance_types=features, # list of distance types to replace in the dataset. these should match the keys of the dict passed into the evaluator.
        task_col_name=task_col_name, # name of the task language column in your dataset
        transfer_col_name=transfer_col_name, # name of the transfer language column in your dataset
        iso_conversion=tasks[task][1] # indicate whether to convert lang codes in your dataset from ISO to Glottocode using the file in self.iso_map_file
    )
    # run LangRank and evaluate task performance
    score = evaluator.evaluate(
        dataset_path=f'data/{task}_updated.csv',
        features=features + ['geographic', 'genetic'], # list of columns in the dataset CSV to use for evaluation
        performance_col_name=tasks[task][0], # name of the column in the dataset CSV containing task performance scores
        task_col_name=task_col_name,
        transfer_col_name=transfer_col_name
    )
    print(f'Task: {task} NDCG: {score}')

Task: mt NDCG: 31.507795242646075
Task: dep NDCG: 70.98819315587522
Task: el NDCG: 67.23559357664477
Task: pos NDCG: 26.49555731440469
Task: taxi1500 NDCG: 21.765757909109226
